In [151]:
from sklearn import datasets
import pandas as pd
import numpy as np
from toolz import pipe

In [41]:
from sklearn.linear_model import LinearRegression

# Multivariate Linear Regression

In [3]:
diabetes = datasets.load_diabetes()

In [37]:
X = pd.DataFrame(diabetes.data)
Y = pd.DataFrame(diabetes.target)
X.columns = diabetes.feature_names
Y.columns = ['y']

In [38]:
X.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641


In [40]:
pd.DataFrame(diabetes)

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641
...,...,...,...,...,...,...,...,...,...,...
437,0.041708,0.050680,0.019662,0.059744,-0.005697,-0.002566,-0.028674,-0.002592,0.031193,0.007207
438,-0.005515,0.050680,-0.015906,-0.067642,0.049341,0.079165,-0.028674,0.034309,-0.018118,0.044485
439,0.041708,0.050680,-0.015906,0.017282,-0.037344,-0.013840,-0.024993,-0.011080,-0.046879,0.015491
440,-0.045472,-0.044642,0.039062,0.001215,0.016318,0.015283,-0.028674,0.026560,0.044528,-0.025930


这里尝试先尝试用用一元线性回归

In [59]:
reg = LinearRegression().fit(X.iloc[:,0:1],Y)

In [63]:
reg.coef_

array([[304.18307453]])

In [64]:
reg.intercept_

array([152.13348416])

In [105]:
I = np.linspace(1,1,X.shape[0])
y = np.array(Y.y)
x_1 = np.array(X.iloc[:,0])

In [106]:
residual_0 = y - (np.dot(y,I)/(np.dot(I,I)))*I

In [107]:
np.dot(I,residual_0) # 可以发现 I 与 residual_0正交

-1.3642420526593924e-12

接下来将Residual_0 投影至 $X_1$上，我们得到

In [108]:
residual_1 = residual_0 - ((np.dot(residual_0,x_1))/(np.dot(x_1,x_1)))*x_1

In [170]:
np.dot(x_1,residual_1)

-3.552713678800501e-15

我们发现x_1与residual_1是正交的

In [114]:
print((np.dot(y,I)/(np.dot(I,I))))
print(((np.dot(residual_0,x_1))/(np.dot(x_1,x_1))))

152.13348416289594
304.18307452829487


而且不难发现，上述投影的系数，就是我们需要的回归系数。上述流程被称为Schmit正交化，也可看作矩阵的"QR"分解。

QR decompositon: Suppose $X$ is a matrix, then there exists a orthonormal matrix $Q$ and an unpper triangular matrix $R$, such that for
$$
X = QR.
$$
If $X$ is the data matrix, then we have
\begin{eqnarray}
\hat \beta & = &R^{-1}Q^\prime \mathbf y \\
\hat{\mathbf{y}} &=& QQ^\prime \mathbf y
\end{eqnarray}

Or the OLS implies that 
$$
\hat \beta = (X^\prime X)^{-1}X^\prime\mathrm y
$$

In [123]:
X = np.array(X)
X = np.c_[I_n,X]

In [130]:
Q,R = np.linalg.qr(X)

In [166]:
np.dot(np.dot(np.linalg.inv(R),Q.transpose()),y)
np.linalg.inv(R).dot(Q.transpose()).dot(y) # 或者更快了，属于是

array([ 152.13348416,  -10.01219782, -239.81908937,  519.83978679,
        324.39042769, -792.18416163,  476.74583782,  101.04457032,
        177.06417623,  751.27932109,   67.62538639])

In [167]:
# 或者我们还可以用OLS得到系数的估计值
np.linalg.inv(X.transpose().dot(X)).dot(X.transpose()).dot(yb)

array([ 152.13348416,  -10.01219782, -239.81908937,  519.83978679,
        324.39042769, -792.18416163,  476.74583782,  101.04457032,
        177.06417623,  751.27932109,   67.62538639])

In [183]:
np.dot(X[:,1],X[:,2])

0.17373710056366723

In [175]:
reg = LinearRegression().fit(X,y)

In [165]:
reg.intercept_

152.1334841628965

In [164]:
reg.coef_

array([   0.        ,  -10.01219782, -239.81908937,  519.83978679,
        324.39042769, -792.18416163,  476.74583782,  101.04457032,
        177.06417623,  751.27932109,   67.62538639])

md，这个数据太特殊了，推导出现问题都没发现。